In [1]:
%reset -f

In [2]:
from sssom.io import parse_sssom_table
import pandas as pd

### Annotate

In [3]:
!runoak -i sqlite:obo:mondo annotate "Marfan syndrome" --matches-whole-text

object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome

---
object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome


### Lexmatch

In [4]:
%%time
!runoak -i sqlite:obo:mondo lexmatch i^MONDO: i^DOID: -o demo.tsv
!head -n 20 demo.tsv

# curie_map:
#   MONDO: http://purl.obolibrary.org/obo/MONDO_
#   oio: http://www.geneontology.org/formats/oboInOwl#
#   owl: http://www.w3.org/2002/07/owl#
#   rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
#   rdfs: http://www.w3.org/2000/01/rdf-schema#
#   semapv: https://w3id.org/semapv/
#   skos: http://www.w3.org/2004/02/skos/core#
#   sssom: https://w3id.org/sssom/
# license: https://w3id.org/sssom/license/unspecified
# mapping_set_id: https://w3id.org/sssom/mappings/1b96dcf2-1b34-48b2-ae8e-85112ae8b556
subject_id	subject_label	predicate_id	object_id	object_label	mapping_justification	mapping_tool	confidence	subject_match_field	object_match_field	match_string
MONDO:0000001	disease or disorder	skos:closeMatch	MONDO:0043543	iatrogenic disease	semapv:LexicalMatching	oaklib	0.5	oio:hasDbXref	oio:hasDbXref	icd9:799.9
MONDO:0000004	adrenocortical insufficiency	skos:closeMatch	MONDO:0007071	adrenocortical hypofunction, chronic primary congenital	semapv:LexicalMatching	oaklib	0.5	oio:

### Relationships

In [5]:
!runoak -i sqlite:obo:mondo relationships  "Marfan syndrome" -o demo.tsv
!head -n 20 demo.tsv


In [6]:
# RO:0004003 => "has material basis in germline mutation in"
!runoak -i sqlite:obo:mondo relationships  --predicates RO:0004003 -o demo.tsv
!head -n 20 demo.tsv
